<a href="https://colab.research.google.com/github/Mughal72/Book_Website/blob/main/Traininig_Rec_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

# Load the book dataset
book_df = pd.read_csv("/content/books.csv", encoding='latin-1')

# Load the user dataset
user_df = pd.read_csv("/content/users.csv")

# Prepare the data for Surprise library
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(user_df[['User-ID', 'Genres', 'Ratings']], reader)

# Split the data into train and test sets (not used in this case)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Create an empty dictionary to store models for each user
user_models = {}

# Train a separate model for each user
for user_id in user_df['User-ID'].unique():
    user_data = Dataset.load_from_df(user_df.loc[user_df['User-ID'] == user_id, ['User-ID', 'Genres', 'Ratings']], reader)
    trainset_user = user_data.build_full_trainset()
    model_user = KNNBasic(k=5, min_k=1)
    model_user.fit(trainset_user)
    user_models[user_id] = model_user

# Get book recommendations for each user using their respective models
for user_id in user_df['User-ID'].unique():
    user_items = user_df.loc[user_df['User-ID'] == user_id, 'Genres']
    user_unrated_books = [book for book in book_df['Genres'] if book not in user_items]
    model_user = user_models[user_id]
    predictions = [model_user.predict(user_id, book) for book in user_unrated_books]
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

    # Print the recommended books for the user
    print("Top 10 recommended books for User", user_id)
    for rec in recommendations:
        book_title = book_df.loc[book_df['Genres'] == rec.iid, 'Book-Title'].values[0]
        print(f"- {book_title} (Estimated rating: {rec.est})")
    print('\n')


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [1]:
pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811673 sha256=0cff9de76962c1ffc8e795f7735699a655c919d5ea8ede31ba05feac16625bea
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


we still iterate over each unique user ID in the user dataset. However, for each user, we create a separate dataset (user_data) containing only their own ratings. We then build a new collaborative filtering model (model_user) based on this user-specific dataset.

By doing this, we can generate personalized recommendations for each user, taking into account their own ratings and preferences. The recommendations will vary for each user, as the model is trained and evaluated on their specific data.

In [ ]:
# another test for recommendation uing CF

In [2]:
pip install scikit-learn pandas

In [3]:
pip install surprise

# Method - 2 Using Matrix Factorization (SVD)

In [4]:
pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 14.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

# Load the book dataset and user dataset into DataFrames
books_df = pd.read_csv('/content/books.csv', encoding='latin-1')
users_df = pd.read_csv('/content/users.csv')

users_df = users_df.drop_duplicates(subset='User-ID')



# Create the user-item matrix
user_item_matrix = users_df.pivot(index='User-ID', columns='ISBN', values='Ratings').fillna(0)
user_item_matrix_sparse = csr_matrix(user_item_matrix.values)

# Apply matrix factorization using SVD
svd = TruncatedSVD(n_components=9)
user_factors = svd.fit_transform(user_item_matrix_sparse)
item_factors = svd.components_.T

# Generate recommendations for a given user
user_id = "amber"  # Example user ID
user_index = users_df[users_df['User-ID'] == user_id].index[0]  # Get the index of the user
user_vector = user_factors[user_index]  # User vector for the given user
scores = np.dot(item_factors, user_vector)  # Compute scores by dot product
top_n = 5  # Number of recommendations to generate
top_items = np.argsort(scores)[-top_n:]  # Get the indices of top items
recommended_books = books_df.iloc[top_items]  # Get the recommended books from the original book dataset

# Print the recommended books
print("Recommended books for User", user_id, ":")
print(recommended_books)


Recommended books for User amber :
                                          Book-Title  \
5  Reinforcement Learning: An Introduction (Adapt...   
6  The Master Algorithm: How the Quest for the Ul...   
7  Hands-On Machine Learning with Scikit-Learn an...   
8            Machine Learning for Absolute Beginners   
0         Artificial Intelligence: A Modern Approach   

                             Author-Name  \
5  Richard S. Sutton and Andrew G. Barto   
6                         Pedro Domingos   
7                         Aurélien Géron   
8                        Oliver Theobald   
0        Stuart Russell and Peter Norvig   

                                         Description         Level  \
5  A classic textbook on reinforcement learning, ...      advanced   
6  A popular science book that explores the histo...     beginners   
7  A practical guide to machine learning using Py...  intermediate   
8  A friendly introduction to machine learning fo...     beginners   
0  The leadin

# Method-3

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF

# Load the book dataset and user dataset into DataFrames
books_df = pd.read_csv('/content/books.csv', encoding='latin-1')
users_df = pd.read_csv('/content/users.csv')

# Remove duplicate entries in the user dataset
users_df = users_df.drop_duplicates(subset=['User-ID', 'ISBN'])

# Create the user-item matrix
user_item_matrix = users_df.pivot(index='User-ID', columns='ISBN', values='Ratings').fillna(0)

# Determine the appropriate number of components for NMF
n_components = min(user_item_matrix.shape)  # Number of components matches the matrix dimensions

# Apply matrix factorization using Non-negative Matrix Factorization (NMF)
nmf = NMF(n_components=n_components)
user_factors = nmf.fit_transform(user_item_matrix)
item_factors = nmf.components_

# Generate recommendations for a given user
user_id = '19sw125'  # Example user ID
user_index = users_df[users_df['User-ID'] == user_id].index[0]  # Find the index of the user
user_vector = user_factors[user_index]  # User vector for the given user
scores = np.dot(item_factors.T, user_vector)  # Compute scores by dot product
top_n = 5  # Number of recommendations to generate
top_indices = np.argsort(scores)[::-1][:top_n]  # Get the indices of top-N items
recommended_books = books_df.loc[top_indices, 'Book-Title']

# Print the recommended books
print("Recommended books for User", user_id, ":")
print(recommended_books)

Recommended books for User 19sw125 :
2    Artificial Intelligence: A Modern Approach 3rd...
8              Machine Learning for Absolute Beginners
7    Hands-On Machine Learning with Scikit-Learn an...
6    The Master Algorithm: How the Quest for the Ul...
5    Reinforcement Learning: An Introduction (Adapt...
Name: Book-Title, dtype: object
